并发: 假同时 一段时间内同时处理多个任务  单核也可以并发

并行: 真同时 同时处理多个任务  必须要多核

Python中实现并发的手段有哪些？

主流操作系统:

* 线程
* 进程

主流的语言通常提供用户空间的调度： 协程

## 线程

In [1]:
import threading

In [2]:
def worker():
    print('work')

In [3]:
thread = threading.Thread(target=worker)  # 创建线程对象 target参数是一个函数， 这个函数即线程要执行的逻辑

In [4]:
thread.start() # start 方法启动一个线程， 当这个线程的逻辑执行完毕的时候，线程自动退出, Python 没有提供主动退出线程的方法

work


所以一定要注意线程的退出

In [6]:
import time

def worker(num):
    time.sleep(1)
    print('worker-{}'.format(num))
    

In [7]:
for x in range(5):
    t = threading.Thread(target=worker, args=(x, ))
    t.start()

worker-0worker-1

worker-2
worker-3worker-4



如何标识一个线程

In [10]:
threading.current_thread()  # 返回当前线程

<_MainThread(MainThread, started 140664094971712)>

In [12]:
threading.Thread(target=lambda:print(threading.current_thread())).start()

<Thread(Thread-10, started 140663182055168)>


In [13]:
thread = threading.current_thread()

In [16]:
thread.is_alive()

True

### logging

In [33]:
import logging
import importlib

importlib.reload(logging)

<module 'logging' from '/home/comyn/.pyenv/versions/3.5.2/lib/python3.5/logging/__init__.py'>

In [36]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')

In [37]:
logging.warning('haha')

2017-03-11 09:37:08,980 WARNING MainThread haha


In [38]:
def worker(num):
    logging.warning('worker-{}'.format(num))

In [39]:
for x in range(5):
    t = threading.Thread(target=worker, args=(x, ))
    t.start()

2017-03-11 09:39:10,743 WARNING Thread-16 worker-0
2017-03-11 09:39:10,744 WARNING Thread-17 worker-1
2017-03-11 09:39:10,745 WARNING Thread-18 worker-2
2017-03-11 09:39:10,745 WARNING Thread-19 worker-3
2017-03-11 09:39:10,746 WARNING Thread-20 worker-4


通常会用logging来替代print

### 参数

In [40]:
def add(x, y):
    logging.info(x + y)

In [41]:
add(1, 2)

2017-03-11 09:43:05,490 INFO MainThread 3


In [42]:
add(x=1, y=2)

2017-03-11 09:43:15,870 INFO MainThread 3


In [44]:
threading.Thread(target=add, args=(1, 2)).start()

2017-03-11 09:43:51,371 INFO Thread-22 3


In [46]:
threading.Thread(target=add, kwargs={'x':1, 'y':2}).start()

2017-03-11 09:44:35,228 INFO Thread-23 3


In [47]:
threading.Thread(target=add, args=(1, ), kwargs={'y': 2}).start()

2017-03-11 09:45:04,394 INFO Thread-24 3


通过args参数传递位置参数， 通过kwargs传递关键字参数

### 控制线程名字

In [48]:
threading.Thread(target=add, args=(1, 2), name='add').start()

2017-03-11 09:46:58,173 INFO add 3


通过name参数控制线程名字

In [49]:
def worker():
    logging.info('starting')
    time.sleep(2)
    logging.info('completed')

In [50]:
t1 = threading.Thread(target=worker, name='worker')
t2 = threading.Thread(target=worker, name='worker')

In [51]:
t1.start()
t2.start()

2017-03-11 09:51:41,392 INFO worker starting
2017-03-11 09:51:41,394 INFO worker starting
2017-03-11 09:51:43,398 INFO worker completed
2017-03-11 09:51:43,401 INFO worker completed


线程可以重名, 线程名并不是线程的唯一标识，但是通常应该避免线程重名，通常的处理手段是加前缀

In [53]:
t1 == t2

False

### daemon 与 non-daemon

In [62]:
t = threading.Thread(target=worker, daemon=True)

In [63]:
t.start()

2017-03-11 10:34:00,259 INFO Thread-27 starting
2017-03-11 10:34:02,267 INFO Thread-27 completed


In [64]:
t = threading.Thread(target=worker)

In [65]:
t.start()

2017-03-11 10:34:25,974 INFO Thread-28 starting
2017-03-11 10:34:27,987 INFO Thread-28 completed


线程退出时,其daemon子线程也会退出， 而non-daemon子线程不会退出

In [66]:
threading.current_thread().is_alive()

True

线程退出会等待所有的non-daemon子线程退出

join方法会阻塞直到线程退出或者超时, timeout 是可选的，如果不设置timeout， 会一直等待线程退出

In [67]:
threading.enumerate() # 获取所有线程

[<_MainThread(MainThread, started 140664094971712)>,
 <Thread(Thread-2, started daemon 140663735711488)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140663702140672)>,
 <Heartbeat(Thread-3, started daemon 140663727318784)>,
 <ParentPollerUnix(Thread-1, started daemon 140663693223680)>]

In [68]:
for t in threading.enumerate():
    print(t.name)

MainThread
Thread-2
IPythonHistorySavingThread
Thread-3
Thread-1


通过实例化Thread类

In [69]:
class MyThread(threading.Thread):
    def run(self):
        logging.info('run')

In [70]:
t = MyThread()

In [71]:
t.start()

2017-03-11 11:11:31,127 INFO Thread-29 run


Python中通常不使用这种方法

In [72]:
t.run()

2017-03-11 11:12:23,761 INFO MainThread run


In [76]:
t = threading.Thread(target=worker)

In [78]:
t.run()

AttributeError: _target

In [77]:
t.start()

2017-03-11 11:13:54,060 INFO Thread-31 starting
2017-03-11 11:13:56,069 INFO Thread-31 completed


如果不是以继承的方式创建线程， run方法和start方法只能执行其中一个

### thread local

In [81]:
ctx = threading.local()

In [82]:
ctx.data = 5

In [83]:
ctx.data

5

In [86]:
data = 'abc'

In [88]:
def worker():
    logging.info(data)
    logging.info(ctx.data)

In [89]:
worker()

2017-03-11 11:18:51,450 INFO MainThread abc
2017-03-11 11:18:51,453 INFO MainThread 5


In [91]:
threading.Thread(target=worker).start()

2017-03-11 11:20:03,536 INFO Thread-33 abc
Exception in thread Thread-33:
Traceback (most recent call last):
  File "/home/comyn/.pyenv/versions/3.5.2/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/comyn/.pyenv/versions/3.5.2/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-88-c46dda5d9c3f>", line 3, in worker
    logging.info(ctx.data)
AttributeError: '_thread._local' object has no attribute 'data'



thread local 对象的属性， 只在当前线程可见

### 定时器/延迟执行

In [95]:
def worker():
    logging.info('run')

In [96]:
t = threading.Timer(interval=5, function=worker)

In [97]:
t.start()

2017-03-11 11:34:40,438 INFO Thread-35 run


In [98]:
t.is_alive()

False

In [100]:
t = threading.Timer(interval=5, function=worker)

In [101]:
t.name = 'worker'

In [103]:
t.daemon = True

In [104]:
t.start()

2017-03-11 11:37:07,010 INFO worker run


In [115]:
t = threading.Timer(interval=300, function=worker)

In [116]:
t.name = 'worker'

In [117]:
t.start()

In [119]:
t.cancel()

In [120]:
for x in threading.enumerate():
    print(x.name)

MainThread
Thread-2
IPythonHistorySavingThread
Thread-3
Thread-1


In [121]:
t.is_alive()

False

In [122]:
def worker():
    logging.info('starting')
    time.sleep(30)
    logging.info('completed')

In [123]:
t = threading.Timer(interval=0, function=worker)

In [124]:
t.start()

2017-03-11 11:43:37,557 INFO Thread-41 starting


In [125]:
t.cancel()

In [126]:
t.is_alive()

True

2017-03-11 11:44:07,564 INFO Thread-41 completed


当function参数所指定函数开始执行的时候， cancel无效

## 同步

In [128]:
import random
import datetime

In [139]:
def worker(event: threading.Event):
    s = random.randint(1, 5)
    #time.sleep(s)
    event.wait(s)
    event.set()
    logging.info('sleep {}'.format(s))
    

def boss(event: threading.Event):
    start = datetime.datetime.now()
    event.wait()
    logging.info('worker exit {}'.format(datetime.datetime.now() - start))




In [130]:
event = threading.Event()

In [131]:
event.set()

In [132]:
event.wait()

True

wait会阻塞线程直到set方法被调用或者超时

In [141]:
def start():
    event = threading.Event()
    b = threading.Thread(target=boss, args=(event, ), name='boss')
    b.start()
    for x in range(5):
        threading.Thread(target=worker, args=(event, ), name='worker-{}'.format(x)).start()

In [142]:
start()

2017-03-11 14:26:08,193 INFO worker-2 sleep 1
2017-03-11 14:26:08,194 INFO boss worker exit 0:00:01.002193
2017-03-11 14:26:08,194 INFO worker-1 sleep 3
2017-03-11 14:26:08,194 INFO worker-3 sleep 4
2017-03-11 14:26:08,194 INFO worker-0 sleep 5
2017-03-11 14:26:08,195 INFO worker-4 sleep 3


event可以在线程之间发送信号

通常用于某个线程需要等待其他线程处理完成某些动作之后才能启动

In [143]:
event = threading.Event()

In [144]:
event.wait(1)

False

In [145]:
def worker(event: threading.Event):
    while not event.wait(3):
        logging.info('run run run')

In [146]:
event = threading.Event()

In [147]:
threading.Thread(target=worker, name='printer', args=(event, )).start()

2017-03-11 14:37:42,351 INFO printer run run run
2017-03-11 14:37:45,353 INFO printer run run run
2017-03-11 14:37:48,356 INFO printer run run run
2017-03-11 14:37:51,360 INFO printer run run run


In [148]:
event.set()

In [149]:
event.is_set()

True

In [150]:
event.clear()

In [151]:
event.is_set()

False

In [152]:
def worker(event: threading.Event):
    while not event.is_set():
        # biz
        pass

In [157]:
class Timer:
    def __init__(self, interval, function, *args, **kwargs):
        self.interval = interval
        self.function = function
        self.args = args
        self.kwargs = kwargs
        self.event = threading.Event()
        self.thread = threading.Thread(target=self.__target)
    
    def __target(self):
        if not self.event.wait(self.interval):
            self.function(*self.args, **self.kwargs)
    
    def start(self):
        self.thread.start()
    
    def cancel(self):
        self.event.set()

In [154]:
def worker():
    logging.info('run')

In [158]:
t = Timer(interval=5, function=worker)

In [159]:
t.start()

2017-03-11 15:07:23,525 INFO Thread-43 run


event 用于线程之间发送信号

### lock

In [160]:
class Counter:
    def __init__(self):
        self.__val = 0
    
    @property
    def value(self):
        return self.__val
    
    def inc(self):
        self.__val += 1 # self.__val = self.__val + 1
    
    def dec(self):
        self.__val -= 1 # self.__val = self.__val - 1

In [161]:
counter = Counter()

In [168]:
def fn():
    if random.choice([-1, 1]) > 0:
        logging.info('inc')
        counter.inc()
    else:
        logging.info('dec')
        counter.dec()

In [169]:
for x in range(10):
    threading.Thread(target=fn).start()

2017-03-11 15:15:55,083 INFO Thread-54 dec
2017-03-11 15:15:55,100 INFO Thread-55 inc
2017-03-11 15:15:55,104 INFO Thread-56 inc
2017-03-11 15:15:55,105 INFO Thread-57 inc
2017-03-11 15:15:55,105 INFO Thread-58 dec
2017-03-11 15:15:55,106 INFO Thread-59 dec
2017-03-11 15:15:55,106 INFO Thread-60 inc
2017-03-11 15:15:55,107 INFO Thread-61 inc
2017-03-11 15:15:55,109 INFO Thread-62 dec
2017-03-11 15:15:55,110 INFO Thread-63 inc


In [171]:
counter.value  # 不知道的

3

In [172]:
lock = threading.Lock()

In [173]:
lock.acquire()

True

In [174]:
lock.acquire()

KeyboardInterrupt: 

对于lock实例，只能调用一次acquire方法， 再次调用会被阻塞， 直到release方法被调用

In [175]:
lock.release()

In [176]:
lock.acquire()

True

In [182]:
class Counter:
    def __init__(self):
        self.__val = 0
        self._lock = threading.Lock()
    
    @property
    def value(self):
        with self._lock:
            return self.__val
    
    def inc(self):
        with self._lock:
            self.__val += 1 # self.__val = self.__val + 1
    
    def dec(self):
        with self._lock:
            self.__val -= 1 # self.__val = self.__val - 1

In [183]:
counter = Counter()

In [184]:
def fn():
    if random.choice([-1, 1]) > 0:
        logging.info('inc')
        counter.inc()
    else:
        logging.info('dec')
        counter.dec()

In [185]:
for x in range(10):
    threading.Thread(target=fn).start()

2017-03-11 15:37:06,958 INFO Thread-64 inc
2017-03-11 15:37:06,959 INFO Thread-65 inc
2017-03-11 15:37:06,960 INFO Thread-66 dec
2017-03-11 15:37:06,960 INFO Thread-67 inc
2017-03-11 15:37:06,961 INFO Thread-68 dec
2017-03-11 15:37:06,969 INFO Thread-69 inc
2017-03-11 15:37:06,974 INFO Thread-70 inc
2017-03-11 15:37:06,976 INFO Thread-71 inc
2017-03-11 15:37:06,978 INFO Thread-72 dec
2017-03-11 15:37:06,982 INFO Thread-73 dec


In [186]:
counter.value

2

何时需要加锁？

In [187]:
lock = threading.Lock()

In [188]:
lock.acquire()

True

In [191]:
lock.acquire(blocking=False) # 当再次加锁时， 如果blocking为False， 那么并不会阻塞，而是返回False

False

In [193]:
lock.acquire(timeout=3)# 如果blocking为True， timeout >=0 会阻塞到超时，并返回False

False

预先启动10个线程，处理一些任务，当其中一个线程在处理其中一个任务时， 其他线程可以处理其他任务

In [199]:
def worker(tasks):
    for task in tasks:
        if task.lock.acquire(False):
#             try:
            logging.info(task.name)
#             finally:
#                 task.lock.release()

In [195]:
class Task:
    def __init__(self, name):
        self.name = name
        self.lock = threading.Lock()

In [196]:
tasks = [Task(x) for x in range(10)]

In [200]:
for x in range(5):
    threading.Thread(target=worker, args=(tasks, ), name='worker-{}'.format(x)).start()

2017-03-11 16:03:59,289 INFO worker-0 0
2017-03-11 16:03:59,290 INFO worker-1 1
2017-03-11 16:03:59,292 INFO worker-2 2
2017-03-11 16:03:59,315 INFO worker-2 7
2017-03-11 16:03:59,295 INFO worker-4 4
2017-03-11 16:03:59,298 INFO worker-0 5
2017-03-11 16:03:59,302 INFO worker-1 6
2017-03-11 16:03:59,294 INFO worker-3 3
2017-03-11 16:03:59,317 INFO worker-2 8
2017-03-11 16:03:59,319 INFO worker-4 9


锁是不可重入

#### 可重入锁

In [201]:
rlock = threading.RLock()

In [202]:
rlock.acquire()

True

In [203]:
rlock.acquire(False)

True

In [204]:
rlock.release()

In [205]:
rlock.release()

可重入锁在同一个线程内， 可以多次acquire成功，但是只能有一个线程acquire成功， acquire几次，就需要release几次

### Condition

In [2]:
import threading
import logging
import importlib
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')

In [76]:
import random


class Dispatcher:
    def __init__(self):
        self.data = None
        self.event = threading.Event()
        self.cond = threading.Condition()
    
    def consumer(self):
        while not self.event.is_set():
            with self.cond:
                self.cond.wait()
                logging.info(self.data)
    
    def producer(self):
        for _ in range(10):
            data = random.randint(0, 100)
            logging.info(data)
            self.data = data
            with self.cond:
                self.cond.notify()
            self.event.wait(1)
        self.event.set()

In [77]:
d = Dispatcher()

In [78]:
p = threading.Thread(target=d.producer, name='producer')

In [79]:
for x in range(4):
    threading.Thread(target=d.consumer, name='consumer-{}'.format(x)).start()

In [80]:
p.start()

2017-03-11 17:19:32,366 INFO [producer] 95
2017-03-11 17:19:32,371 INFO [consumer-0] 95
2017-03-11 17:19:33,372 INFO [producer] 35
2017-03-11 17:19:33,380 INFO [consumer-1] 35
2017-03-11 17:19:34,384 INFO [producer] 49
2017-03-11 17:19:34,386 INFO [consumer-2] 49
2017-03-11 17:19:35,387 INFO [producer] 99
2017-03-11 17:19:35,391 INFO [consumer-3] 99
2017-03-11 17:19:36,391 INFO [producer] 64
2017-03-11 17:19:36,394 INFO [consumer-0] 64
2017-03-11 17:19:37,395 INFO [producer] 39
2017-03-11 17:19:37,398 INFO [consumer-1] 39
2017-03-11 17:19:38,398 INFO [producer] 18
2017-03-11 17:19:38,401 INFO [consumer-2] 18
2017-03-11 17:19:39,401 INFO [producer] 18
2017-03-11 17:19:39,403 INFO [consumer-3] 18
2017-03-11 17:19:40,404 INFO [producer] 77
2017-03-11 17:19:40,413 INFO [consumer-0] 77
2017-03-11 17:19:41,413 INFO [producer] 33
2017-03-11 17:19:41,415 INFO [consumer-1] 33


Condition 通常用于生产者消费者模式， 生产者生产消息之后， 使用notify 或者 notify_all 通知消费者消费

消费者使用wait方法阻塞等待生产者通知

notify通知指定个wait的线程， notify_all通知所有的wait线程

无论notify/notify_all还是wait 都必须先acqurie， 完成后必须确保release， 通常使用with语法

### Barrier

In [1]:
import threading

In [2]:
import logging
import importlib
importlib.reload(logging)

<module 'logging' from '/home/comyn/.pyenv/versions/3.5.2/lib/python3.5/logging/__init__.py'>

In [3]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')

In [7]:
def worker(barrier: threading.Barrier):
    logging.info('waitting for {} threads'.format(barrier.n_waiting))
    try:
        worker_id = barrier.wait()
        logging.info('after barrier {}'.format(worker_id))
    except threading.BrokenBarrierError:
        logging.warning('aborting')
        

In [8]:
barrier = threading.Barrier(3)

In [10]:
import time

for x in range(3):
    threading.Thread(target=worker, name='worker-{}'.format(x), args=(barrier, )).start()
    time.sleep(0.5)
logging.info('started')

2017-03-12 09:14:29,700 INFO [worker-0] waitting for 0 threads
2017-03-12 09:14:30,206 INFO [worker-1] waitting for 1 threads
2017-03-12 09:14:30,711 INFO [worker-2] waitting for 2 threads
2017-03-12 09:14:30,716 INFO [worker-2] after barrier 2
2017-03-12 09:14:30,716 INFO [worker-0] after barrier 0
2017-03-12 09:14:30,716 INFO [worker-1] after barrier 1
2017-03-12 09:14:31,213 INFO [MainThread] started


凑齐一波线程，才继续往下走

In [11]:
barrier.n_waiting

0

In [12]:
barrier = threading.Barrier(3)

In [13]:
def abort(barrier):
    logging.info('aborting')
    barrier.abort()

In [14]:
for x in range(2):
    threading.Thread(target=worker, name='worker-{}'.format(x), args=(barrier,)).start()

2017-03-12 09:22:06,621 INFO [worker-0] waitting for 0 threads
2017-03-12 09:22:06,622 INFO [worker-1] waitting for 0 threads


In [15]:
barrier.abort()

2017-03-12 09:22:39,260 WARNING [worker-0] aborting
2017-03-12 09:22:39,260 WARNING [worker-1] aborting


当abort方法被执行的时候， wait方法会抛出BrokenBarrierError异常

In [16]:
help(barrier.wait)

Help on method wait in module threading:

wait(timeout=None) method of threading.Barrier instance
    Wait for the barrier.
    
    When the specified number of threads have started waiting, they are all
    simultaneously awoken. If an 'action' was provided for the barrier, one
    of the threads will have executed that callback prior to returning.
    Returns an individual index number from 0 to 'parties-1'.



In [18]:
barrier.reset()

In [19]:
barrier.wait(3)

BrokenBarrierError: 

### Semaphore

In [20]:
s = threading.Semaphore(3)

In [21]:
s.acquire()

True

In [22]:
s.acquire(False)

True

In [23]:
s.acquire(False)

True

In [24]:
s.acquire(False)

False

锁是信号量的特例： 为1的信号量

In [33]:
class Pool:
    def __init__(self, num):
        self.num = num
        self.conns = [self._make_connect(x) for x in range(num)]
        self.s = threading.Semaphore(num)
    
    def _make_connect(self, name):
        return name
    
    def get(self):
        self.s.acquire()
        return self.conns.pop()
    
    def return_resource(self, conn):
        self.conns.insert(0, conn)
        self.s.release()

In [30]:
import random

def worker(pool):
    logging.info('started')
    name = pool.get()
    logging.info('got connect {}'.format(name))
    time.sleep(random.randint(1, 3))
    pool.return_resource(name)
    logging.info('return resource {}'.format(name))

In [34]:
pool = Pool(3)

In [35]:
for x in range(5):
    threading.Thread(target=worker, args=(pool,), name='worker-{}'.format(x)).start()

2017-03-12 10:02:57,065 INFO [worker-0] started
2017-03-12 10:02:57,066 INFO [worker-1] started
2017-03-12 10:02:57,067 INFO [worker-2] started
2017-03-12 10:02:57,069 INFO [worker-0] got connect 2
2017-03-12 10:02:57,070 INFO [worker-4] started
2017-03-12 10:02:57,069 INFO [worker-3] started
2017-03-12 10:02:57,075 INFO [worker-1] got connect 1
2017-03-12 10:02:57,084 INFO [worker-2] got connect 0
2017-03-12 10:02:59,091 INFO [worker-0] return resource 2
2017-03-12 10:02:59,091 INFO [worker-4] got connect 2
2017-03-12 10:03:00,099 INFO [worker-1] return resource 1
2017-03-12 10:03:00,099 INFO [worker-3] got connect 1
2017-03-12 10:03:00,100 INFO [worker-2] return resource 0
2017-03-12 10:03:01,097 INFO [worker-4] return resource 2
2017-03-12 10:03:01,106 INFO [worker-3] return resource 1


信号量也是对资源的保护， 但是和锁不一样的地方在于， 锁限制只有一个线程可以访问共享资源， 而信号量限制指定个线程可以访问共享资源

## 线程之间的通讯

In [36]:
import queue

In [37]:
def producer(queue: queue.Queue, event: threading.Event):
    while not event.wait(3):
        data = random.randint(0, 100)
        logging.info(data)
        queue.put(data)

In [38]:
def consumer(queue: queue.Queue, event: threading.Event):
    while not event.is_set():
        logging.info(queue.get())

In [39]:
q = queue.Queue()

In [40]:
e = threading.Event()

In [41]:
threading.Thread(target=consumer, name='consumer', args=(q, e)).start()

In [43]:
threading.Thread(target=producer, name='producer', args=(q, e)).start()

2017-03-12 10:39:33,514 INFO [producer] 26
2017-03-12 10:39:33,516 INFO [consumer] 26
2017-03-12 10:39:36,516 INFO [producer] 6
2017-03-12 10:39:36,522 INFO [consumer] 6
2017-03-12 10:39:39,523 INFO [producer] 12
2017-03-12 10:39:39,531 INFO [consumer] 12
2017-03-12 10:39:42,532 INFO [producer] 97
2017-03-12 10:39:42,534 INFO [consumer] 97
2017-03-12 10:39:45,535 INFO [producer] 59
2017-03-12 10:39:45,537 INFO [consumer] 59
2017-03-12 10:39:48,538 INFO [producer] 70
2017-03-12 10:39:48,540 INFO [consumer] 70
2017-03-12 10:39:51,541 INFO [producer] 81
2017-03-12 10:39:51,545 INFO [consumer] 81


In [44]:
e.set()

In [45]:
q = queue.Queue(maxsize=5)

In [47]:
for x in range(5):
    q.put(x)

In [50]:
q.put(6) # 阻塞

KeyboardInterrupt: 

In [51]:
q.put(6, False) # raise queue.Full

Full: 

In [52]:
q.put(6, timeout=3) # 阻塞直到timeout， 并抛出 queue.Full

Full: 

In [53]:
q.put_nowait(6) # q.put(6, False)

Full: 

In [60]:
q.get()

KeyboardInterrupt: 

In [61]:
q.get(False) # raise queue.Empty

Empty: 

In [62]:
q.get(timeout=3) # 阻塞直到超时，抛出queue.Empty

Empty: 

In [63]:
q.get_nowait() # q.get(False)

Empty: 

queue.Queue 是线程安全的

全局解释器锁 GIL

内置容器都是线程安全的

In [64]:
import collections

## 多进程

In [65]:
import multiprocessing

In [84]:
def worker():
    logging.info('worker')

In [87]:
p = multiprocessing.Process(target=worker, name='worker')

In [88]:
p.start()

2017-03-12 11:11:13,266 INFO [worker] worker


In [82]:
importlib.reload(logging)

<module 'logging' from '/home/comyn/.pyenv/versions/3.5.2/lib/python3.5/logging/__init__.py'>

In [83]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s [%(processName)s] %(message)s')

In [89]:
p.pid

3834

In [90]:
p.exitcode

0

In [92]:
p.terminate() # 进程可以被杀死

每个进程会启动一个解释器， 所以进程的代价比线程要高

线程所有的同步方式， 在进程上都有， 而且用法一样

**但是通讯方式不一样**

由于多进程跨解释器的， 所以进程的通讯，数据需要序列化和反序列化

GIL对多进程无效, 内置容器不是进程安全的， queue.Queue 也不是进程安全的

In [95]:
multiprocessing.Queue()

In [96]:
from multiprocessing import Manager

In [97]:
mgr = Manager()

In [99]:
d = mgr.dict()

In [100]:
mgr.list()

<ListProxy object, typeid 'list' at 0x7fe548260128>

In [101]:
ns = mgr.Namespace()

In [102]:
ns.f = 3

多进程环境，尽量避免数据交互

## 多线程和多进程的选择

* CPU密集型用多进程， 可以充分利用CPU
* IO密集型用多线程，减少序列化/反序列化

请求/应答 这种模型， 更多时候是结合使用的

通常由master进程接收请求， 分发给多个worker进程处理， worker中再使用线程来进一步并发处理， 最后，返回结果给master做响应

## concurrent.futures

In [103]:
import concurrent

In [104]:
from concurrent import futures

In [105]:
pool = futures.ThreadPoolExecutor(max_workers=5)

In [107]:
fut = pool.submit(lambda: 1+1)

In [108]:
fut.result()

2

In [110]:
fut.done()

True

In [111]:
def worker():
    time.sleep(30)
    logging.info('worker')

In [112]:
fut = pool.submit(worker)

In [113]:
fut.done()

False

In [114]:
fut.cancel()

False

In [115]:
fut.running()

False

In [116]:
def worker():
    raise Exception('haha')

In [117]:
fut = pool.submit(worker)

In [118]:
fut.exception()

Exception('haha')

In [119]:
futures.ProcessPoolExecutor()

In [120]:
help(futures.ThreadPoolExecutor)

Help on class ThreadPoolExecutor in module concurrent.futures.thread:

class ThreadPoolExecutor(concurrent.futures._base.Executor)
 |  This is an abstract base class for concrete asynchronous executors.
 |  
 |  Method resolution order:
 |      ThreadPoolExecutor
 |      concurrent.futures._base.Executor
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, max_workers=None)
 |      Initializes a new ThreadPoolExecutor instance.
 |      
 |      Args:
 |          max_workers: The maximum number of threads that can be used to
 |              execute the given calls.
 |  
 |  shutdown(self, wait=True)
 |      Clean-up the resources associated with the Executor.
 |      
 |      It is safe to call this method several times. Otherwise, no other
 |      methods can be called after this one.
 |      
 |      Args:
 |          wait: If True then shutdown will not return until all running
 |              futures have finished executing and the resources used by the
 | 

In [121]:
help(futures._base.Executor)

Help on class Executor in module concurrent.futures._base:

class Executor(builtins.object)
 |  This is an abstract base class for concrete asynchronous executors.
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, exc_type, exc_val, exc_tb)
 |  
 |  map(self, fn, *iterables, timeout=None, chunksize=1)
 |      Returns an iterator equivalent to map(fn, iter).
 |      
 |      Args:
 |          fn: A callable that will take as many arguments as there are
 |              passed iterables.
 |          timeout: The maximum number of seconds to wait. If None, then there
 |              is no limit on the wait time.
 |          chunksize: The size of the chunks the iterable will be broken into
 |              before being passed to a child process. This argument is only
 |              used by ProcessPoolExecutor; it is ignored by
 |              ThreadPoolExecutor.
 |      
 |      Returns:
 |          An iterator equivalent to: map(func, *iterables) but the call